# Data Download and Exploration

This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [1]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_pums_2017_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

In [3]:
from src.data import data_collection

This line may take as long as 10-20 minutes depending on your network connection and computer specs

In [4]:
data_collection.download_data_and_load_into_sql()

Successfully created database and all tables

Successfully downloaded ZIP file
    https://www2.census.gov/programs-surveys/acs/data/pums/2017/5-Year/csv_pwa.zip
    
Successfully downloaded GZIP file
    https://lehd.ces.census.gov/data/lodes/LODES7/wa/wac/wa_wac_S000_JT00_2017.csv.gz
    
Successfully downloaded GZIP file
    https://lehd.ces.census.gov/data/lodes/LODES7/wa/wa_xwalk.csv.gz
    
Successfully downloaded CSV file
    https://www2.census.gov/geo/docs/maps-data/data/rel/2010_Census_Tract_to_2010_PUMA.txt
    
Successfully loaded CSV file into `pums_2017` table
        
Successfully loaded CSV file into `puma_names_2010` table
        
Successfully loaded CSV file into `wa_jobs_2017` table
        
Successfully loaded CSV file into `wa_geo_xwalk` table
        
Successfully loaded CSV file into `ct_puma_xwalk` table
        


Now it's time to explore the data!

In [2]:
import psycopg2
import pandas as pd

In [3]:
DBNAME = "opportunity_youth"

In [4]:
conn = psycopg2.connect(dbname=DBNAME)

In [19]:
pd.read_sql("""SELECT puma, sum(pwgtp) AS "Number of OY in SKC "  FROM pums_2017 WHERE puma between '11610' and '11615' and agep between '16' and '24' and sch = '1' and (esr = '6' or esr = '3') GROUP BY puma""", conn)

,puma,Number of OY in SKC
0,11611,2038.0
1,11612,1977.0
2,11613,2006.0
3,11614,1530.0
4,11615,1210.0


Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

In [35]:
pd.read_sql("""SELECT CONCAT(statefp,  countyfp, tractce), puma5ce, *  FROM ct_puma_xwalk WHERE statefp = '53'""", conn)

,concat,puma5ce,statefp,countyfp,tractce,puma5ce
0,53001950100,10600,53,001,950100,10600
1,53001950200,10600,53,001,950200,10600
2,53001950300,10600,53,001,950300,10600
3,53001950400,10600,53,001,950400,10600
4,53001950500,10600,53,001,950500,10600
...,...,...,...,...,...,...
1453,53077940002,10902,53,077,940002,10902
1454,53077940003,10902,53,077,940003,10902
1455,53077940004,10902,53,077,940004,10902
1456,53077940005,10902,53,077,940005,10902


In [33]:
pd.read_sql("""
SELECT w_geocode GEOID,c000 Jobs,ca01 Job_Under29 
FROM wa_jobs_2017 
JOIN 
""", conn)

,geoid,jobs,job_under29
0,530019501001010,1,0


In [63]:
pd.read_sql("""SELECT * FROM puma_names_2010 WHERE state_name = 'Washington' LIMIT 5 """, conn)

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name
0,53,Washington ...,1030,10100,5310100,G53010100,Whatcom County--Bellingham City ...
1,53,Washington ...,1031,10200,5310200,G53010200,"Skagit, Island & San Juan Counties ..."
2,53,Washington ...,1032,10300,5310300,G53010300,Chelan & Douglas Counties ...
3,53,Washington ...,1032,10400,5310400,G53010400,"Stevens, Okanogan, Pend Oreille & Ferry Counti..."
4,53,Washington ...,1032,10600,5310600,G53010600,"Whitman, Asotin, Adams, Lincoln, Columbia & Ga..."


In [71]:
# sum_by = 'w_geocode'
pd.read_sql(f"""SELECT geo.bgrp  "GEOID",SUM(c000)  "SALL", SUM(ca01) "SU29", puma.puma5ce "PUMA", name.puma_name
                FROM wa_jobs_2017 jobs
                JOIN wa_geo_xwalk geo
                ON jobs.w_geocode = geo.tabblk2010
                JOIN ct_puma_xwalk puma
                ON geo.trct = CONCAT(puma.statefp, puma.countyfp, puma.tractce)
                JOIN puma_names_2010 name
                ON puma.puma5ce = name.puma
                WHERE puma5ce between '11610' and '11615'
                GROUP BY geo.bgrp, "PUMA",name.puma_name """, conn)

,statefp,countyfp,tractce,puma5ce
0,53,001,950100,10600
1,53,001,950200,10600
2,53,001,950300,10600
3,53,001,950400,10600
4,53,001,950500,10600
...,...,...,...,...
1453,53,077,940002,10902
1454,53,077,940003,10902
1455,53,077,940004,10902
1456,53,077,940005,10902


In [83]:
pd.read_sql("""SELECT DISTINCT(puma5ce) "PUMA", statefp, countyfp  FROM ct_puma_xwalk WHERE statefp = '53' and countyfp = '033' """, conn)

,PUMA,statefp,countyfp
0,11601,53,033
1,11602,53,033
2,11603,53,033
3,11604,53,033
4,11605,53,033
5,11606,53,033
6,11607,53,033
7,11608,53,033
8,11609,53,033
9,11610,53,033


Make sure you close the DB connection when you are done using it

In [9]:
conn.close()